In [ ]:
import cv2
import numpy as np
import cvzone
import pickle
import math
from pyfirmata import Arduino, SERVO
from time import sleep
import threading

interval = 0.1


port = 'COM6'
servo_base = 3
servo_esquerdo = 4
servo_direito = 5
servo_garra = 6

board = Arduino(port)

board.digital[servo_base].mode=SERVO
board.digital[servo_esquerdo].mode=SERVO
board.digital[servo_direito].mode=SERVO
board.digital[servo_garra].mode=SERVO

framewidth = 640
frameheigh = 480

posBase = []
posEsquerdo = []
posDireito = []
posGarra = []
posGeral = [posBase,posEsquerdo,posDireito, posGarra]


def rotateservo(servo,angle):
    board.digital[servo].write(angle)
    sleep(0.015)
    

def routine(servo, angle):

    if servo == 3:
        pos = posGeral[0]
    elif servo == 4:
        pos = posGeral[1]
    elif servo == 5:
        pos = posGeral[2]
    elif servo == 6:
        pos = posGeral[3]
    
    if len(pos) > 0:
        qtt = len(pos)
            #print(qtt)
        beforeAngle = pos[qtt-1]
        #print('angulo anterior: ',beforeAngle, ' - novo angulo: ', angle)
        if beforeAngle > angle:

            for i in range(beforeAngle,angle,-1):
                rotateservo(servo,i)
        elif beforeAngle < angle:
            for i in range(beforeAngle,angle):
                rotateservo(servo,i)
        else:
            pass
    else:
        #print('angulo: ',angle)
        for i in range(0,angle):
            rotateservo(servo,i)
        
    pos.append(angle)
    sleep(interval)


    
cap = cv2.VideoCapture(1)
cap.set(3, framewidth)
cap.set(4, frameheigh)

def empty(a):
    pass


def gradient(pt1, pt2):
    try:
        z = ((pt2[1]-pt1[1])/(pt2[0]-pt1[0]))
        return z
    except ZeroDivisionError:
        z = 0
        return z


def detectReposition(imgContour, imgDilateCropped):
    pRect1, pRect2 = 171, 249
    pFRect1, pFRect2 = 174 + 35, 253 + 35
    

    imgRectangle = imgDilateCropped[pRect2:pFRect2, pRect1:pFRect1]
    
    contours, hierarchy = cv2.findContours(imgRectangle, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    count = 0 
    
    cv2.rectangle(imgContour, (pRect1, pRect2),(pFRect1, pFRect2),(0,0,255),2)
    
    for cnt in contours:
        count = cv2.countNonZero(imgRectangle)
        if count > 400 and count < 800:
            cv2.rectangle(imgContour, (pRect1, pRect2),(pFRect1, pFRect2),(0,255,0),2)
            return True
  

def distance(x1, y1, x2, y2):

    return (((x2 - x1)**2 +(y2 - y1)**2)**0.5)
 

def getContours(imgDil, imgContour, img, estadoCaixa):
    
    contours, hierarchy = cv2.findContours(imgDil, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    count = 0
    
    imgGrayCropped = cv2.cvtColor(imgContour, cv2.COLOR_BGR2GRAY)
    imgBlurCropped = cv2.GaussianBlur(imgGrayCropped, (3,3), 1)
    imgThresholdCropped = cv2.adaptiveThreshold(imgBlurCropped,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                                     cv2.THRESH_BINARY_INV, 25, 14)            
    imgMedianCropped = cv2.medianBlur(imgThresholdCropped,5)
    kernelCropped = np.ones((3,3), np.uint8)
    imgDilateCropped = cv2.dilate(imgMedianCropped, kernelCropped, iterations = 1)
    
    
    #[190, 105], [344, 105], [193, 296]
    p1, p2 = 190, 105
    pg1, pg2 = 193, 296
    cv2.line(img, (p1, p2) , (pg1, pg2), (255, 0, 0), 2)
    
    detectReposition(imgContour, imgDilateCropped)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 6000 and area < 12000:
            count = count + 1
            #cv2.drawContours(imgContour, cnt, -1, (255, 0, 255), 1)
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
            x , y, w , h = cv2.boundingRect(approx)
            
            imgCrop = imgDilateCropped[y:y+h, x:x+w]
            count = cv2.countNonZero(imgCrop)
            if count < estadoCaixa:
                color = (0,255,0)
                thickness = 5
                cvzone.putTextRect(imgContour,"Vazio",(x,y+h-10), scale = 1, thickness=2, offset=0, colorR=color)
                
                mdx = int(((x+x+w)/2))
                mdy = int(((y+y+h)/2))
                
                #Pega a posição em graus da caixa vazia
                pointsList = [[p1, p2], [mdx, mdy], [pg1, pg2]]
                
                #Linha reta para tomar como base onde o robô está
                cv2.line(img, (p1, p2), (mdx, mdy), (255, 0, 0), 2)
                
                #Mede a Distância da Linha
                dist = int(distance(p1, p2, mdx, mdy))
                
                if dist >= 160:
                    dist = 160
                elif dist <=30:
                    dist = 0
                    
                pt1, pt2, pt3 = pointsList[-3:]
                m1 = gradient(pt1, pt2)
                m2 = gradient(pt1, pt3)
                angR = math.atan((m2-m1)/(1+(m2*m1))) #ANGLE IN RADIOUS
                angD = round(math.degrees(angR)) # ANGLE IN DEGREES
                cv2.putText(img,str(angD),(pt2[0]+46,pt2[1]+20),cv2.FONT_HERSHEY_COMPLEX,
                                               1,(255,0,0),2)
                
                anguloBase = abs(angD) + 85
            
            else:
                color = (0,0,255)
                thickness = 3
                cvzone.putTextRect(imgContour,"Preenchido",(x,y+h-10), scale = 1, thickness=2, offset=0, colorR=color)
            
            cv2.rectangle(imgContour, (x, y), (x + w , y + h), color, thickness)
            
            #cv2.line(img, (378, 291), (int((x+w)/1.5), int(y+h)), (0, 0, 255), 5)
    
w_, h_ = 600, 410
crpW, crpH = 130, 0

def robotMovements(anguloBase, dist):
    sleep(2)
    #Posição Inicial
    routine(servo_base, 80)
    routine(servo_esquerdo, 90)
    routine(servo_direito, 130)
    routine(servo_garra, 54)

    #preparar pra pegar objeto
    routine(servo_base, 80)
    routine(servo_esquerdo, 135)
    routine(servo_direito, 119)
    routine(servo_garra, 54)

    #Pegar o Objeto
    routine(servo_base, 80)
    routine(servo_esquerdo, 135)
    routine(servo_direito, 119)
    routine(servo_garra, 140)    

    #Peparar para giro
    routine(servo_base, 75)
    routine(servo_esquerdo, 92)
    routine(servo_direito, 160)
    routine(servo_garra, 140)  

    #Giro
    routine(servo_base, anguloBase)
    routine(servo_esquerdo, 92)
    routine(servo_direito, 160)                   
    routine(servo_garra, 140) 

    #Preparar pra soltar peça
    routine(servo_base, anguloBase)
    routine(servo_esquerdo, (dist))#-25
    routine(servo_direito, 130)
    routine(servo_garra, 140) 

    #Soltar peça
    routine(servo_base, anguloBase)
    routine(servo_esquerdo, (dist))#-25
    routine(servo_direito, 130)                    
    routine(servo_garra, 43) 

    #Preparar pra voltar pra posição inicial
    routine(servo_base, anguloBase)
    routine(servo_esquerdo, 98)
    routine(servo_direito, 168)                    
    routine(servo_garra, 145) 

    #Posição Inicia
    routine(servo_base, 80)
    routine(servo_esquerdo, 90)
    routine(servo_direito, 130)
    routine(servo_garra, 124) 
    
def getContoursAndRunRobot(imgDil, imgContour, img, estadoCaixa):
    
    contours, hierarchy = cv2.findContours(imgDil, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    count = 0
    
    imgGrayCropped = cv2.cvtColor(imgContour, cv2.COLOR_BGR2GRAY)
    imgBlurCropped = cv2.GaussianBlur(imgGrayCropped, (3,3), 1)
    imgThresholdCropped = cv2.adaptiveThreshold(imgBlurCropped,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                                     cv2.THRESH_BINARY_INV, 25, 14)            
    imgMedianCropped = cv2.medianBlur(imgThresholdCropped,5)
    kernelCropped = np.ones((3,3), np.uint8)
    imgDilateCropped = cv2.dilate(imgMedianCropped, kernelCropped, iterations = 1)
    
    
    #[190, 105], [344, 105], [193, 296]
    p1, p2 = 190, 105
    pg1, pg2 = 193, 296
    cv2.line(img, (p1, p2) , (pg1, pg2), (255, 0, 0), 2)

    detectReposition(imgContour, imgDilateCropped)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 6000 and area < 12000:
            count = count + 1
            #cv2.drawContours(imgContour, cnt, -1, (255, 0, 255), 1)
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
            x , y, w , h = cv2.boundingRect(approx)
            
            imgCrop = imgDilateCropped[y:y+h, x:x+w]
            count = cv2.countNonZero(imgCrop)
            if count < estadoCaixa:
                color = (0,255,0)
                thickness = 5
                cvzone.putTextRect(imgContour,"Vazio",(x,y+h-10), scale = 1, thickness=2, offset=0, colorR=color)
                
                mdx = int(((x+x+w)/2))
                mdy = int(((y+y+h)/2))
                
                #Pega a posição em graus da caixa vazia
                pointsList = [[p1, p2], [mdx, mdy], [pg1, pg2]]
                
                #[[189, 108], [345, 105], [187, 271]]
                
                #Linha reta para tomar como base onde o robô está
                cv2.line(img, (p1, p2), (mdx, mdy), (255, 0, 0), 2)
                
                #Mede a Distância da Linha
                dist = int(distance(p1, p2, mdx, mdy))
                
                dist = int(dist/1.3)
                #if dist >= 160:
                #    dist = 160
                #elif dist <=30:
                #    dist = 0
                    
                pt1, pt2, pt3 = pointsList[-3:]
                m1 = gradient(pt1, pt2)
                m2 = gradient(pt1, pt3)
                angR = math.atan((m2-m1)/(1+(m2*m1))) #ANGLE IN RADIOUS
                angD = round(math.degrees(angR)) # ANGLE IN DEGREES
                cv2.putText(img,str(angD),(pt2[0]+46,pt2[1]+20),cv2.FONT_HERSHEY_COMPLEX,
                                               1,(255,0,0),2)
                
                anguloBase = abs(angD) + 85
                
                if detectReposition(imgContour, imgDilateCropped) == True:
                    robotMovements(anguloBase, dist)
                    break
            else:
                color = (0,0,255)
                thickness = 3
                cvzone.putTextRect(imgContour,"Preenchido",(x,y+h-10), scale = 1, thickness=2, offset=0, colorR=color)
            
            cv2.rectangle(imgContour, (x, y), (x + w , y + h), color, thickness)
            

def createBars():
    
    cv2.namedWindow('Parameters')
    cv2.createTrackbar('Threshold1', 'Parameters', 55, 255,empty)
    cv2.createTrackbar('Threshold2', 'Parameters', 46, 255,empty)
    cv2.createTrackbar('estadoCaixa', 'Parameters', 1584, 1800,empty)

def processingCamera():
    createBars()
    
    #Posição Inicial
    routine(servo_base, 80)
    routine(servo_esquerdo, 90)
    routine(servo_direito, 130)
    routine(servo_garra, 140)
    
    while True:
        success, img = cap.read()
        estadoCaixa = cv2.getTrackbarPos('estadoCaixa', 'Parameters')
        #threshold1 = cv2.getTrackbarPos('Threshold1', 'Parameters')
        #threshold2 = cv2.getTrackbarPos('Threshold2', 'Parameters')
        
        height, width, channels = img.shape
        img = img[crpH:crpH+h_, crpW:crpW+w_]
        imgContour = img.copy()
        roi = img

        imgBlur = cv2.GaussianBlur(img, (7, 7), 1)
        imgGray = cv2.cvtColor(imgBlur, cv2.COLOR_BGR2GRAY)
        #imgCanny = cv2.Canny(imgGray, threshold1, threshold2)
        imgCanny = cv2.Canny(imgGray, 55, 46)
        kernel = np.ones((5, 5))
        imgDil = cv2.dilate(imgCanny, kernel, iterations=1)    

        getContours(imgDil, roi, img, estadoCaixa)    

        cv2.imshow("Webcam", img)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

def controlRobot():
    while True:
        success, img = cap.read()        
        height, width, channels = img.shape
        img = img[crpH:crpH+h_, crpW:crpW+w_]
        imgContour = img.copy()
        roi = img

        imgBlur = cv2.GaussianBlur(img, (7, 7), 1)
        imgGray = cv2.cvtColor(imgBlur, cv2.COLOR_BGR2GRAY)
        imgCanny = cv2.Canny(imgGray, 55, 46)
        kernel = np.ones((5, 5))
        imgDil = cv2.dilate(imgCanny, kernel, iterations=1)    

        getContoursAndRunRobot(imgDil, roi, img, 1500)    

        #cv2.imshow("Webcam", img)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break    
    
    
t = threading.Thread(target=processingCamera)
t.start()

r = threading.Thread(target=controlRobot)
r.start()
